In [36]:
import sys
sys.path.append('..')

import os
from sqlalchemy import create_engine, text
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon, MultiPoint
from shapely import wkb
from geopandas.tools import sjoin

from utilities_amigocloud import AmigocloudFunctions

In [37]:
from config import RUTA_UNIDAD_ONE_DRIVE
from config import RUTA_LOCAL_ONE_DRIVE
from config import API_AMIGOCLOUD_TOKEN_ADM
from config import POSTGRES_UTEA

RUTA_COMPLETA = os.path.join(RUTA_UNIDAD_ONE_DRIVE, RUTA_LOCAL_ONE_DRIVE)
PATH_CAT = RUTA_UNIDAD_ONE_DRIVE + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Pulverizacion\2025\SHP\catastro_S09_MIERCOLES.shp'
PATH_XLSX_GRUPOS = RUTA_UNIDAD_ONE_DRIVE + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AVANCE COSECHA\2025\DATA\GRUPO_COSECHA.xlsx'

In [38]:
def obtener_engine():
    return create_engine(
        f"postgresql+psycopg2://{POSTGRES_UTEA['USER']}:{POSTGRES_UTEA['PASSWORD']}@{POSTGRES_UTEA['HOST']}:{POSTGRES_UTEA['PORT']}/{POSTGRES_UTEA['DATABASE']}"
    )

def obtener_planificacion():
    engine = obtener_engine()
    try:
        query = """
            SELECT *
            FROM drones_pulverizacion.planificacion_pulv
            WHERE procesado=false;
        """
        gdf = gpd.read_postgis(query, engine, geom_col='geom')
        return gdf
    except Exception as e:
        print(f"Error en la consulta: {e}")
        return gpd.GeoDataFrame()
    return None

def marcar_como_procesado(id_os):
    engine = obtener_engine()  # tu función que crea el engine
    try:
        with engine.begin() as conn:
            query = text("""
                UPDATE drones_pulverizacion.planificacion_pulv
                SET procesado = true
                WHERE id = :id_os
            """)
            conn.execute(query, {"id_os": id_os})
            print(f"✔️ id {id_os} marcado como procesado.")
    except Exception as e:
        print(f"❌ Error al actualizar: {e}")
    return None

def obtener_parte_diario_por_os(ors):
    engine = obtener_engine()
    try:
        query = f"""
            SELECT * FROM drones_pulverizacion.parte_diario_pulv where os = {ors}
        """
        gdf = gpd.read_postgis(query, engine, geom_col='geom')
        return gdf
    except Exception as e:
        print(f"❌ Error en la consulta: {e}")
        return gpd.GeoDataFrame()
    return None

def convertir_a_multipolygon(geometry):
    if isinstance(geometry, Polygon):
        return MultiPolygon([geometry])
    return geometry

def convertir_a_wkb(polygon):
    wkb_data = wkb.dumps(polygon, hex=True)
    return wkb_data

def cargar_a_amigocloud(gdf):
    # repreyectar a WGS84
    gdf_pla_gral = gdf.to_crs(epsg=4326)
    # convertir poligonos a multipoligonos
    gdf_pla_gral['geom'] = gdf_pla_gral['geom'].apply(convertir_a_multipolygon)
    
    gdf_pla_gral['unidad_01'] = gdf_pla_gral['unidad_01'].astype(int)
    gdf_pla_gral['unidad_03'] = gdf_pla_gral['unidad_03'].astype(int)
    gdf_pla_gral['os'] = gdf_pla_gral['os'].astype(int)
    gdf_pla_gral['soca'] = gdf_pla_gral['soca'].astype(int)
    gdf_pla_gral['id'] = gdf_pla_gral['id'].astype(int)
    gdf_pla_gral['inst'] = gdf_pla_gral['inst'].astype(int)
    gdf_pla_gral['mezcla'] = gdf_pla_gral['mezcla'].astype(int)
    
    # recorrer el gdf de lotes y cargarlo a amigocloud
    id_proyecto = 35248
    for index, row in gdf_pla_gral.iterrows():
        wkb_hex = convertir_a_wkb(row['geom'])
        insert_sql = f"""
            INSERT INTO dataset_360912 (id, unidad_01, unidad_02, unidad_03, unidad_04, unidad_05, area, os, mezcla, geometry)
            VALUES ({row['id']}, {row['unidad_01']}, '{row['unidad_02']}', {row['unidad_03']}, '{row['unidad_04']}', '{row['unidad_05']}', {row['area']}, '{row['os']}', '{row['mezcla']}', ST_SetSRID(ST_GeomFromWKB('\\x{wkb_hex}'), 4326));
        """
        query_sql = {'query': insert_sql}
        resultado_post = amigocloud.ejecutar_query_sql(id_proyecto, insert_sql, 'post')
        print(f'Lote registrado en AmigoCloud')
    return None

def get_catastro():
    engine = obtener_engine()
    try:
        query = f'''
            SELECT * FROM drones_control_bio.catastro_drones
        '''
        gdf = gpd.read_postgis(query, engine, geom_col='geom')
        return gdf
    except Exception as e:
        print(f"❌ Error al obtener la capa de catastro: {e}")
        return gpd.GeoDataFrame()
    return None

In [39]:
amigocloud = AmigocloudFunctions(token=API_AMIGOCLOUD_TOKEN_ADM)
amigocloud

In [40]:
gdf_cat = get_catastro()
gdf_plan = obtener_planificacion()
gdf_plan

,id,geom,codigo_canero,nombre_canero,procesado,mezcla,obs
0,79,"MULTIPOINT (460520.780 8079811.414, 460579.452...",42305,JALDIN LIDIA DURAN DE,False,70,None
1,81,MULTIPOINT (487245.623 8085777.322),12190,RODRIGUEZ LOPEZ PEDRO,False,66,None
2,82,"MULTIPOINT (487251.979 8084746.660, 487180.595...",12190,RODRIGUEZ LOPEZ PEDRO,False,66,None
3,73,"MULTIPOINT (487463.257 8102367.039, 487140.564...",10697,PAZ REA JAIME EDUARDO,False,65,None
4,74,"MULTIPOINT (483976.707 8095409.095, 484166.411...",4974,GUTIERREZ GIL MARCIAL,False,67,None
5,75,"MULTIPOINT (479311.777 8086270.651, 479130.873...",18086,SUAREZ CUELLAR JENNY SONIA,False,68,None
6,76,"MULTIPOINT (478729.952 8083441.709, 478771.999...",18086,SUAREZ CUELLAR JENNY SONIA,False,68,None
7,77,"MULTIPOINT (475051.252 8094715.427, 475223.355...",13058,SUAREZ DEMIQUEL DAVID,False,65,None
8,78,"MULTIPOINT (471781.297 8096948.853, 471304.103...",13171,SUAREZ JUSTINIANO DAVID,False,65,None
9,80,"MULTIPOINT (486635.929 8085981.205, 486722.958...",12190,RODRIGUEZ LOPEZ PEDRO,False,66,None


# CARGAR PARTE DIARIO

In [41]:
gdf_intersect_result = gpd.sjoin(gdf_cat, gdf_plan, how='inner')
os_para_cargar = list(set(gdf_intersect_result['id_right']))
os_para_cargar

[73, 74, 75, 76, 77, 78, 79, 80, 81, 82]

In [42]:
gdf_intersect_result['mezcla']

1919     65
2525     65
2528     65
2580     65
2590     65
2608     65
2654     65
4980     65
5093     65
5830     70
5858     65
5968     65
5986     65
6203     70
6210     70
6820     65
6942     66
7484     65
8670     67
8672     67
8673     67
8804     65
8809     65
8811     65
8920     65
9004     65
9014     68
9024     68
9037     65
9119     68
10088    65
10089    65
10129    65
12818    65
12820    65
13044    65
13048    65
13052    65
13053    65
13056    65
13058    65
13059    65
13064    65
13075    65
13128    65
13522    68
13523    68
13526    68
13531    68
13551    68
13692    66
13732    66
13733    66
13734    66
13735    66
Name: mezcla, dtype: int64

In [43]:
gdf_result

,geom,unidad_01,unidad_02,unidad_03,unidad_04,unidad_05,area,os,soca,inst,mezcla
1919,"MULTIPOLYGON (((489819.168 8103463.008, 489818...",47,SANTA MARTHA,10697,PAZ REA JAIME EDUARDO,A11,5.365206,73,1,69,65
2525,"MULTIPOLYGON (((488072.068 8102965.544, 488084...",47,SANTA MARTHA,10697,PAZ REA JAIME EDUARDO,B3.1,7.144949,73,1,69,65
2528,"MULTIPOLYGON (((488208.279 8103125.052, 488321...",47,SANTA MARTHA,10697,PAZ REA JAIME EDUARDO,B3.2,2.387886,73,0,69,65
2580,"MULTIPOLYGON (((487272.027 8102609.752, 487492...",47,SANTA MARTHA,10697,PAZ REA JAIME EDUARDO,C6.2,19.764522,73,0,69,65
2590,"MULTIPOLYGON (((489806.435 8103323.949, 489862...",47,SANTA MARTHA,10697,PAZ REA JAIME EDUARDO,A12.3,8.836473,73,0,69,65
2608,"MULTIPOLYGON (((486984.385 8102840.556, 487451...",47,SANTA MARTHA,10697,PAZ REA JAIME EDUARDO,B7.3,10.445319,73,0,69,65
2654,"MULTIPOLYGON (((489977.794 8102996.315, 489961...",47,SANTA MARTHA,10697,PAZ REA JAIME EDUARDO,A12.2,4.260786,73,1,69,65
4980,"MULTIPOLYGON (((475417.487 8094924.975, 475426...",1395,LA CANDELARIA--SUAREZ DAVID,13058,SUAREZ DEMIQUEL DAVID,L1,25.505972,77,1,62,65
5093,"MULTIPOLYGON (((488616.298 8103763.516, 488519...",47,SANTA MARTHA,10697,PAZ REA JAIME EDUARDO,A7,10.696703,73,1,69,65
5830,"MULTIPOLYGON (((460336.404 8079412.998, 460273...",1605,EL ARROYO--JALDIN,42305,JALDIN LIDIA DURAN DE,L2,10.898405,79,3,131,70


In [44]:
gdf_result = gdf_intersect_result[['geom', 'unidad_01', 'unidad_02', 'codigo_canero', 'nombre_canero', 'unidad_05', 'area', 'id_right', 'soca', 'zona', 'mezcla']].copy()
gdf_result['area'] = gdf_result.geometry.area / 10000
gdf_result.rename(columns={
    'geometry': 'geom',
    'codigo_canero': 'unidad_03',
    'nombre_canero': 'unidad_04',
    'id_right': 'os',
    'zona': 'inst'
}, inplace=True)

gdf_result = gdf_result.set_geometry("geom")
gdf_result['unidad_01'] = gdf_result['unidad_01'].astype(int)
gdf_result['unidad_03'] = gdf_result['unidad_03'].astype(int)
gdf_result['os'] = gdf_result['os'].astype(int)
gdf_result['soca'] = gdf_result['soca'].astype(int)
gdf_result['inst'] = gdf_result['inst'].astype(int)
gdf_result['mezcla'] = gdf_result['mezcla'].astype(int)

In [45]:
gdf_result

,geom,unidad_01,unidad_02,unidad_03,unidad_04,unidad_05,area,os,soca,inst,mezcla
1919,"MULTIPOLYGON (((489819.168 8103463.008, 489818...",47,SANTA MARTHA,10697,PAZ REA JAIME EDUARDO,A11,5.365206,73,1,69,65
2525,"MULTIPOLYGON (((488072.068 8102965.544, 488084...",47,SANTA MARTHA,10697,PAZ REA JAIME EDUARDO,B3.1,7.144949,73,1,69,65
2528,"MULTIPOLYGON (((488208.279 8103125.052, 488321...",47,SANTA MARTHA,10697,PAZ REA JAIME EDUARDO,B3.2,2.387886,73,0,69,65
2580,"MULTIPOLYGON (((487272.027 8102609.752, 487492...",47,SANTA MARTHA,10697,PAZ REA JAIME EDUARDO,C6.2,19.764522,73,0,69,65
2590,"MULTIPOLYGON (((489806.435 8103323.949, 489862...",47,SANTA MARTHA,10697,PAZ REA JAIME EDUARDO,A12.3,8.836473,73,0,69,65
2608,"MULTIPOLYGON (((486984.385 8102840.556, 487451...",47,SANTA MARTHA,10697,PAZ REA JAIME EDUARDO,B7.3,10.445319,73,0,69,65
2654,"MULTIPOLYGON (((489977.794 8102996.315, 489961...",47,SANTA MARTHA,10697,PAZ REA JAIME EDUARDO,A12.2,4.260786,73,1,69,65
4980,"MULTIPOLYGON (((475417.487 8094924.975, 475426...",1395,LA CANDELARIA--SUAREZ DAVID,13058,SUAREZ DEMIQUEL DAVID,L1,25.505972,77,1,62,65
5093,"MULTIPOLYGON (((488616.298 8103763.516, 488519...",47,SANTA MARTHA,10697,PAZ REA JAIME EDUARDO,A7,10.696703,73,1,69,65
5830,"MULTIPOLYGON (((460336.404 8079412.998, 460273...",1605,EL ARROYO--JALDIN,42305,JALDIN LIDIA DURAN DE,L2,10.898405,79,3,131,70


In [46]:
# GARANTIZAR LA INST DEL CAÑERO
# cargar grupos de cosecha
df_grupos = pd.read_excel(PATH_XLSX_GRUPOS, sheet_name='CODIGOS')
df_grupos = df_grupos[df_grupos['INSTITUCION'].notna()]
df_grupos['CODIGO CAÑERO'] = df_grupos['CODIGO CAÑERO'].astype(int)
df_grupos['INSTITUCION'] = df_grupos['INSTITUCION'].astype(int)

# Crear un diccionario de mapeo: {codigo_cañero: institucion}
mapa_institucion = df_grupos.set_index('CODIGO CAÑERO')['INSTITUCION'].to_dict()
# Reemplazar los valores de 'inst' en gdf_result usando el diccionario
# a partir del diccionario, busca el codigo cañero (unidad_03), y reempalza la institucion
gdf_result['inst'] = gdf_result['unidad_03'].map(mapa_institucion)

In [47]:
gdf_result

,geom,unidad_01,unidad_02,unidad_03,unidad_04,unidad_05,area,os,soca,inst,mezcla
1919,"MULTIPOLYGON (((489819.168 8103463.008, 489818...",47,SANTA MARTHA,10697,PAZ REA JAIME EDUARDO,A11,5.365206,73,1,69,65
2525,"MULTIPOLYGON (((488072.068 8102965.544, 488084...",47,SANTA MARTHA,10697,PAZ REA JAIME EDUARDO,B3.1,7.144949,73,1,69,65
2528,"MULTIPOLYGON (((488208.279 8103125.052, 488321...",47,SANTA MARTHA,10697,PAZ REA JAIME EDUARDO,B3.2,2.387886,73,0,69,65
2580,"MULTIPOLYGON (((487272.027 8102609.752, 487492...",47,SANTA MARTHA,10697,PAZ REA JAIME EDUARDO,C6.2,19.764522,73,0,69,65
2590,"MULTIPOLYGON (((489806.435 8103323.949, 489862...",47,SANTA MARTHA,10697,PAZ REA JAIME EDUARDO,A12.3,8.836473,73,0,69,65
2608,"MULTIPOLYGON (((486984.385 8102840.556, 487451...",47,SANTA MARTHA,10697,PAZ REA JAIME EDUARDO,B7.3,10.445319,73,0,69,65
2654,"MULTIPOLYGON (((489977.794 8102996.315, 489961...",47,SANTA MARTHA,10697,PAZ REA JAIME EDUARDO,A12.2,4.260786,73,1,69,65
4980,"MULTIPOLYGON (((475417.487 8094924.975, 475426...",1395,LA CANDELARIA--SUAREZ DAVID,13058,SUAREZ DEMIQUEL DAVID,L1,25.505972,77,1,62,65
5093,"MULTIPOLYGON (((488616.298 8103763.516, 488519...",47,SANTA MARTHA,10697,PAZ REA JAIME EDUARDO,A7,10.696703,73,1,69,65
5830,"MULTIPOLYGON (((460336.404 8079412.998, 460273...",1605,EL ARROYO--JALDIN,42305,JALDIN LIDIA DURAN DE,L2,10.898405,79,3,131,70


In [48]:
# agregar nuevos registros a la base de datos
gdf_result.to_postgis("parte_diario_pulv", obtener_engine(), schema="drones_pulverizacion", if_exists="append")

In [49]:
os_para_cargar

[73, 74, 75, 76, 77, 78, 79, 80, 81, 82]

In [50]:
# CARGAR A AMIGOCLOD
for i in os_para_cargar:
    plan_os = obtener_parte_diario_por_os(i)
    cargar_a_amigocloud(plan_os)
    marcar_como_procesado(i)

Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
✔️ id 73 marcado como procesado.
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
✔️ id 7